# MIP Assignment
## Problem definition
A company delivers products in the city of Valencia, from a set of production facilities located outside the city, to a set of in each of the districts of the city, referred to as demand regions. Your purchase department has identified a set of candidate locations for intermediate warehouse facilities. Your objective is to design the distribution network that minimises distribution costs, from production facilities to intermediate warehouses and from warehouses to regions, for a given demand period in months, and for the different product types. 
The costs are estimated based on the transportation distance and consequently, are the same for each product.

Given the following indices: 
- **$i$**: Production plants $i \in [1, ..., n]$
- **$j$**: Warehouse location $j \in [1, ..., n]$
- **$k$**: District or demand region $k \in [1, ..., n]$
- **$l$**: Product type $l \in [1, ..., n]$
- **$t$**: time period (month) $t \in [1, ..., n]$


And the following data: 
- **$c_{il}$**: Capacity of production plant $i$ for product type $l$
- **$d_{klt}$**: Demand of product $l$ in region $k$ in period $t$
- **$a_{ij}$**: Transportation costs from production plant $i$ to warehouse $j$
- **$b_{jk}$**: Transportation costs from warehouse $j$ to region $k$

Let us define the following decision variables: 
- **$Y_{j}$**: (Binary) {1 if warehouse j is used to deliver to any region at any period, 0 otherwise}
- **$S_{i,j,l,t}$**: (Integer) number of units of product l delivered from factory i to warehouse j at period t. 
- **$T_{j,k,l,t}$**: (Integer) number of units of product l delivered from warehouse j to region k at period t 

The problem is subject to the following constraints:
- **Production Capacity**: The amount delivered from production plants must not exceed the production capacity for each type of product at any given period. 
-**Demand**: The amount delivered from warehouses to regions must satisfy the demand for all product types and all periods
- **Flow**: The units that arrive to a warehouse from all production plants must be equal to the units that leave to all regions at any given period and for each product type

**a**. Write the MIP problem that minimises distribution costs.


## <span style="color: purple"> MODEL DEFINITION

Since our goal is to design a distribution network that minimises the distribution cost per month we formulate the following model:

### Objective function
$$\color{purple} {min Z = (\sum_{i}\sum_{j} ( a_{i,j}* S_{i,j,l,t}))+ (\sum_{i}\sum_{j}( b_{i,j}* T_{j,k,l,t})}$$

### Constraints

##### 1. Production Capacity

$$\color{purple} {S_{i,j,l,t} < C_{i,f}}$$

##### 2. Demand

$$ \color{purple} {T_{j,k,l,t} = d_{k,l,t}} $$

##### 3. Flow


$$ \color{purple} {S_{i,j,l,t} = T_{j,k,l,t}} $$


 As part of a development team, you need to complete parts of the PuLP model that solves this optimization problem. 
 The following cell contains the data required to load the model: 
 

In [1]:
import pandas as pd
import pulp
import os

# Load data into dataframe
plants_df = pd.read_csv('Datasets/plants.csv')
demand_df = pd.read_csv('Datasets/demand.csv')
capacities_df = pd.read_csv('Datasets/plant_capacities.csv')
warehouses_df = pd.read_csv('Datasets/warehouses.csv')
costs_PW_df = pd.read_csv('Datasets/costs_PW.csv')
costs_WR_df = pd.read_csv('Datasets/costs_WR.csv')

plants = plants_df['plant_id'].unique()                  # This will be our index i
regions = demand_df['region_id'].unique()                 # This will be our index k
products = demand_df['product_type'].unique()             # This will be our index l
warehouses = warehouses_df['warehouse_id'].unique()       # This will be our index j
periods = demand_df['date'].unique()                      # This will be our index t

# Set the index in the dataframe and fecth the demand. In the expressions, we will use it as d[k, l, t]
demand_df_indexed = demand_df.set_index(['region_id', 'product_type', 'date'])
filled_demand_index = pd.MultiIndex.from_product([regions, products, periods], names=['regions', 'products', 'periods'])
filled_demand = pd.DataFrame(0, index=filled_demand_index, columns=['demand'])
filled_demand['demand'] = demand_df_indexed['Quantity']
d = filled_demand['demand'].fillna(0)

# Set the index in the dataframe and fetch the capacities. In the expressions, we will use the capacities as c[i,l]
capacities_df_indexed = capacities_df.set_index(['plants', 'products'])
c = capacities_df_indexed['capacity']

# Define the transportation costs from production plants to warehouses
# Set the index in the dataframe and fetch the distribution costs from production plants to warehouses,
# we will use it as a[i.j]
costs_PW_df_indexed = costs_PW_df.set_index(['plant_id', 'warehouse_id'])
a = costs_PW_df_indexed['distance']

# Define the transportation costs from warehouses to regions
# Set the index in the dataframe and fetch the distribution costs from production plants to warehouses,
# we will use it as b[j,k]
costs_WR_df_indexed = costs_WR_df.set_index(['warehouse_id', 'region_id'])
b = costs_WR_df_indexed['distance']


# A very large number of products
M = 99999999999999999

# Instantiate the model
model = pulp.LpProblem("Transport Planning", pulp.LpMinimize)

# binary { 1 if a warehouse j is built, 0 otherwise }
Y = pulp.LpVariable.dicts("Y",
                          [j for j in warehouses],
                          lowBound=0,
                          cat='Binary')


# units of product transported from plant i to warehouse j in period t
S = pulp.LpVariable.dicts("S",
                          [(i, j, l, t) for i in plants for j in warehouses for l in products for t in periods],
                          lowBound=0,
                          cat='Integer')

# units of product l transported from warehouse j to region k in period t
T = pulp.LpVariable.dicts("T",
                          [(j, k, l, t) for j in warehouses for k in regions for l in products for t in periods],
                          lowBound=0,
                          cat='Integer')

/Users/fer112233/PycharmProjects/MIP Notebook Task/ven/lib/python3.8/site-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


You need to write a Python function that calculates the transportation costs from production warehouses to production plants. Use the function that calculates the costs from production plants to warehouses as a reference. Fill in the code of the function in the corresponding line:

In [2]:
def transportation_costs_pw():

    return pulp.lpSum([
        a[i, j] * S[i, j, l, t]
        for i in plants for j in warehouses for l in products for t in periods])

def transportation_costs_wr():
    """
            Returns the sum product of warehouses - regions distribution costs and decision variables

            Args:
                None

            Returns: An PuLP expression with the distribution costs from warehouses to regions
            """

    return pulp.lpSum([
        b[i, j] * T[i, j, l, t]
        for i in plants for j in warehouses for l in products for t in periods
    ])
    # Write your code here


# from this functions, we define the objective function
model += transportation_costs_pw() + transportation_costs_wr(), "Distribution Costs"

Now, you need to complete the definition of the constraints. You are given some of the constraints as an example. Fill in the missing constraint in the following cell.

In [3]:
# subject to:
# The production capacities must be met in all periods:
for i in plants:
    for l in products:
        for t in periods:
            model += pulp.lpSum([
                S[i, j, l, t]
            for j in warehouses]) <= c[i,l], "Capacity " + str((i, l, t))

# The demand for all products in all regions and in all periods must be satisfied
for k in regions:
    for l in products:
        for t in periods:
            model += pulp.lpSum([
                T[j, k, l, t]
            for j in warehouses]) == d[k, l, t], "Demand " + str((k, l, t))

# The amount of each product which arrives to a warehouse should be equal to the amount which exit from that warehouse
# in one period
for j in warehouses:
    for l in products:
        for t in periods:
            model += pulp.lpSum([
                S[i, j, l, t]
                for i in plants]) == pulp.lpSum([
                T[j, k, l, t]
                for k in regions]), "Flow " + str((j, l, t))


# A warehouse will supply a region only when the amount transported of all products in all periods from such warehouse to such region is nonzero
for j in warehouses:
        model += pulp.lpSum([
            T[j, k, l, t]
            for k in regions for l in products for t in periods]) <= M*Y[j], "Logic constraint warehouse " + str((j, k))


The following cell solves the model:

In [4]:
model.solve()
print(pulp.LpStatus[model.status])
print(pulp.value(model.objective))

Optimal
21428721831.0



And the following cells load the decision variables of the solution into dataframes:


In [5]:
# Create a dataframe from the dictionary Y
Y_df = pd.DataFrame.from_dict(Y, orient="index",
                           columns=["Y"], dtype=object)
# Load the solution using a lambda function:
Y_df["Solution"] = Y_df["Y"].apply(lambda item: item.varValue)

# Create a dataframe from dictionary S, 
S_df = pd.DataFrame.from_dict(S, orient="index",
                           columns=["S"], dtype=object)
# Load the solution using a lambda function:
S_df["Solution"] = S_df["S"].apply(lambda item: item.varValue)
# Create index and reindex dataframe
s_idx = pd.MultiIndex.from_product([plants, warehouses, products, periods], names=['plant_id', 'warehouse_id', 'product_type', 'date'])
S_df = S_df.reindex(s_idx)

# Create a dataframe from dictionary T, 

T_df = pd.DataFrame.from_dict(T, orient="index",
                                  columns=["T"], dtype=object)
# Load the solution using a lambda function:
T_df["Solution"] = T_df["T"].apply(lambda item: item.varValue)
# Create index and reindex dataframe
t_idx = pd.MultiIndex.from_product([warehouses, regions, products, periods],  names=['warehouse_id', 'region_id', 'product_type', 'date'])
T_df = T_df.reindex(t_idx)

Use the dataframes S_df and T_df to group the results and and answer the following question, representing the results in the format you find most appropiate 
- What is the maximum number of units of each product type that are sourced from each production site?

Here the Solution column is the sum of units of each product type that were sent from each plant to each warehouse.

In [10]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
S_df.reset_index().groupby(['plant_id', 'warehouse_id', 'product_type']).sum()

Solution
plant_id warehouse_id product_type           
1        1            1                3218.0
                      2              162789.0
                      3                2162.0
                      4                1973.0
                      5                 912.0
                      6                  12.0
                      7                1807.0
                      8               23308.0
                      9                  70.0
         2            1                   0.0
                      2                   0.0
                      3                   0.0
                      4                   0.0
                      5                   0.0
                      6                   0.0
                      7                   0.0
                      8                   0.0
                      9                   0.0
         3            1                   0.0
                      2                   0.0
                      3                   0.0
                      4                   0.0
                      5                   0.0
                      6                   0.0
                      7                   0.0
                      8                   0.0
                      9                   0.0
         4            1                   0.0
                      2                   0.0
                      3                   0.0
                      4                   0.0
                      5                   0.0
                      6                   0.0
                      7                   0.0
                      8                   0.0
                      9                   0.0
         5            1               76727.0
                      2             1821110.0
                      3               21179.0
                      4               45459.0
                      5                4786.0
                      6                2529.0
                      7               39370.0
                      8              331949.0
                      9                4257.0
         6            1                   0.0
                      2                   0.0
                      3                   0.0
                      4                   0.0
                      5                   0.0
                      6                   0.0
                      7                   0.0
                      8                   0.0
                      9                   0.0
         7            1                   0.0
                      2                   0.0
                      3                   0.0
                      4                   0.0
                      5                   0.0
                      6                   0.0
                      7                   0.0
                      8                   0.0
                      9                   0.0
         8            1                   0.0
                      2                   0.0
                      3                   0.0
                      4                   0.0
                      5                   0.0
                      6                   0.0
                      7                   0.0
                      8                   0.0
                      9                   0.0
2        1            1                   0.0
                      2                   0.0
                      3                   0.0
                      4                   0.0
                      5                   0.0
                      6                   0.0
                      7                   0.0
                      8                   0.0
                      9                   0.0
         2            1                   0.0
                      2                   0.0
                      3                   0.0
                      4                   0.0
                      5            

Here the Solution column is the sum of units of each product type that were sent from each warehouse to each region.

In [11]:
T_df.reset_index().groupby(['warehouse_id', 'region_id', 'product_type']).sum()

Solution
warehouse_id region_id product_type           
1            1         1                   0.0
                       2                   0.0
                       3                   0.0
                       4                   0.0
                       5                   0.0
                       6                   0.0
                       7                   0.0
                       8                   0.0
                       9                   0.0
             2         1                   0.0
                       2                   0.0
                       3                   0.0
                       4                   0.0
                       5                   0.0
                       6                   0.0
                       7                   0.0
                       8                   0.0
                       9                   0.0
             3         1                   0.0
                       2                   0.0
                       3                   0.0
                       4                   0.0
                       5                   0.0
                       6                   0.0
                       7                   0.0
                       8                   0.0
                       9                   0.0
             4         1                   0.0
                       2                   0.0
                       3                   0.0
                       4                   0.0
                       5                   0.0
                       6                   0.0
                       7                   0.0
                       8                   0.0
                       9                   0.0
             5         1                   0.0
                       2                   0.0
                       3                   0.0
                       4                   0.0
                       5                   0.0
                       6                   0.0
                       7                   0.0
                       8                   0.0
                       9                   0.0
             6         1                   0.0
                       2                   0.0
                       3                   0.0
                       4                   0.0
                       5                   0.0
                       6                   0.0
                       7                   0.0
                       8                   0.0
                       9                   0.0
             7         1                   0.0
                       2                   0.0
                       3                   0.0
                       4                   0.0
                       5                   0.0
                       6                   0.0
                       7                   0.0
                       8                   0.0
                       9                   0.0
             8         1                   0.0
                       2                   0.0
                       3                   0.0
                       4                   0.0
                       5                   0.0
                       6                   0.0
                       7                   0.0
                       8                   0.0
                       9                   0.0
             9         1                 248.0
                       2               21046.0
                       3                 374.0
                       4                 252.0
                       5                  72.0
                       6                  12.0
                       7                 704.0
                       8                5411.0
                       9                   9.0
             10        1                 226.0
                       2                3571.0
                       3                 33

Carolina García, Fernando Pascual, Ana Alonso, Maria Lloret